## Connect Directories

Import the github repository and move it into the local directory  and remove the defult samples directoy if this has not been done already)


In [0]:
from getpass import getpass
import os

if not os.path.isdir(".git"):
    user = "Boyne272"
    password = getpass('github password')
    os.environ['GITHUB_AUTH'] = user + ':' + password

    # clone the repo
    !git clone https://$GITHUB_AUTH@github.com/msc-acse/acse-8-miniproject-softmax.git repo

    # move the repo up one
    !mv repo/* .
    !mv repo/.git .

    # delete un needed stuff
    !rm -r repo
    !rm -r sample_data/

!git checkout richard
!git pull origin richard
!git log -2

github password··········
Cloning into 'repo'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 633 (delta 13), reused 33 (delta 6), pack-reused 587
Receiving objects: 100% (633/633), 171.79 MiB | 28.68 MiB/s, done.
Resolving deltas: 100% (333/333), done.


Mount the google drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

## Imports

Ipython imports and functions

In [0]:
# ipython setup
!pip install pycm livelossplot
%pylab inline
%load_ext autoreload
%autoreload 2

     |████████████████████████████████| 51kB 1.3MB/s 
     |████████████████████████████████| 491kB 5.5MB/s 
     |████████████████████████████████| 215kB 38.5MB/s 
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 4.5.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3.6, but you'll have coverage 4.5.3 which is incompatible.
  Found existing installation: coverage 3.7.1
    Uninstalling coverage-3.7.1:
      Successfully uninstalled coverage-3.7.1
Populating the interactive namespace from numpy and matplotlib


Module imports

In [0]:
# imports
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from torchvision.transforms import Compose, ToTensor, Normalize, RandomRotation, RandomCrop, ToPILImage, Pad

from livelossplot import PlotLosses
from pycm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# imports from github
from tools import *

Check runtime device

In [0]:
# check running device
device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

Cuda installed! Running on GPU!


## Ensemble of all 4 Models

 ### Load the CSV files of their predictions

In [0]:
import pandas as pd

db = pd.read_csv("/content/models/Comparisons/combined_predictions.csv")

Check the cross corrleation of these matrices

In [0]:
db.corr()

,DAB Drop Aug (98.88%),DAB Drop Aug full (98.2%),DAB opt SGD 2,DAB opt ADAM 1,DAB opt ADAM 2,AlexNet 7 Adam,AlexNet 7 SGD,AlexNet 7 Adam full,AlexNet 7 SGD full,Super Awesome Dodgie thing no pad,Super Awesome Dodgie thing pad,AlexNet 6 wide
DAB Drop Aug (98.88%),1.000000,0.988160,0.987419,0.986022,0.986022,0.987594,0.988542,0.985701,0.984890,0.987295,0.984803,0.984671
DAB Drop Aug full (98.2%),0.988160,1.000000,0.985661,0.983934,0.983934,0.985670,0.986916,0.981631,0.984927,0.985840,0.982116,0.980758
DAB opt SGD 2,0.987419,0.985661,1.000000,0.983237,0.983237,0.984192,0.985791,0.984815,0.982747,0.987268,0.981633,0.979733
DAB opt ADAM 1,0.986022,0.983934,0.983237,1.000000,1.000000,0.984980,0.985189,0.982190,0.984366,0.984201,0.982942,0.980670
DAB opt ADAM 2,0.986022,0.983934,0.983237,1.000000,1.000000,0.984980,0.985189,0.982190,0.984366,0.984201,0.982942,0.980670
AlexNet 7 Adam,0.987594,0.985670,0.984192,0.984980,0.984980,1.000000,0.987954,0.986296,0.985068,0.985386,0.980343,0.983232
AlexNet 7 SGD,0.988542,0.986916,0.985791,0.985189,0.985189,0.987954,1.000000,0.987855,0.987465,0.985860,0.981396,0.984165
AlexNet 7 Adam full,0.985701,0.981631,0.984815,0.982190,0.982190,0.986296,0.987855,1.000000,0.983851,0.984448,0.978563,0.982342
AlexNet 7 SGD full,0.984890,0.984927,0.982747,0.984366,0.984366,0.985068,0.987465,0.983851,1.000000,0.984754,0.979847,0.981868
Super Awesome Dodgie thing no pad,0.987295,0.985840,0.987268,0.984201,0.984201,0.985386,0.985860,0.984448,0.984754,1.000000,0.984643,0.979731


In [0]:
names = db.keys()
names

Index(['DAB Drop Aug (98.88%)', 'DAB Drop Aug full (98.2%)', 'DAB opt SGD 2',
       'DAB opt ADAM 1', 'DAB opt ADAM 2', 'AlexNet 7 Adam', 'AlexNet 7 SGD',
       'AlexNet 7 Adam full', 'AlexNet 7 SGD full',
       'Super Awesome Dodgie thing no pad', 'Super Awesome Dodgie thing pad',
       'AlexNet 6 wide'],
      dtype='object')

### Select the Ensemble

In [0]:
ensemble_names = ["DAB opt SGD 2", "DAB opt ADAM 1", "AlexNet 7 Adam", "AlexNet 7 SGD"]

In [0]:
ensemble = db[ensemble_names]
ensemble

,DAB opt SGD 2,DAB opt ADAM 1,AlexNet 7 Adam,AlexNet 7 SGD
0,2,2,2,2
1,9,9,9,9
2,3,3,3,3
3,8,8,8,8
4,3,3,3,3
5,3,3,3,3
6,8,8,8,8
7,3,3,3,3
8,2,2,2,2
9,5,5,5,5


Combine there predictions with a voting algorithm

In [0]:
output = []

for index, row in ensemble.iterrows():

    # find the unique entries of each row and their duplicates
    uniq, cnt = np.unique(row.values, return_counts=True)
    
    # if there is only one choose it
    if len(uniq) == 1:
        output.append(*uniq)
    
    # if there are two it is a split and pick one at random
    elif cnt[0]==2:
        output.append(np.random.choice(uniq))
        
    # else pick the one with the most agreement
    else:
        output.append(uniq[np.argmax(cnt)])
        
    # there is no case of all 4 models disagreeing
    
print(output)

[2, 9, 3, 8, 3, 3, 8, 3, 2, 5, 6, 3, 3, 3, 1, 5, 4, 8, 6, 3, 7, 5, 7, 5, 7, 0, 3, 5, 4, 4, 6, 8, 5, 9, 5, 6, 6, 3, 0, 6, 4, 7, 8, 1, 1, 6, 9, 0, 5, 0, 1, 1, 4, 8, 4, 1, 3, 3, 3, 4, 6, 4, 8, 3, 6, 5, 3, 4, 6, 1, 5, 9, 9, 7, 3, 2, 3, 7, 6, 3, 4, 1, 1, 5, 9, 6, 5, 1, 0, 1, 2, 7, 1, 6, 7, 3, 6, 5, 9, 2, 0, 8, 8, 2, 3, 5, 9, 1, 0, 8, 5, 2, 0, 0, 5, 5, 2, 9, 0, 9, 3, 6, 9, 5, 6, 2, 0, 7, 6, 2, 9, 0, 4, 1, 3, 4, 6, 9, 9, 5, 8, 4, 5, 9, 6, 8, 2, 4, 7, 3, 5, 1, 8, 5, 8, 6, 3, 2, 7, 6, 2, 5, 6, 7, 5, 2, 8, 0, 6, 6, 4, 1, 9, 5, 8, 6, 5, 6, 8, 7, 4, 4, 0, 6, 5, 7, 2, 6, 2, 1, 8, 9, 0, 7, 4, 3, 0, 4, 6, 3, 1, 3, 5, 0, 8, 8, 5, 8, 2, 2, 3, 2, 7, 5, 7, 5, 4, 5, 4, 7, 2, 3, 7, 5, 8, 9, 0, 7, 5, 0, 1, 6, 6, 3, 0, 9, 8, 4, 9, 0, 2, 0, 3, 0, 0, 6, 8, 3, 7, 9, 4, 4, 0, 5, 5, 9, 6, 4, 0, 4, 2, 7, 3, 2, 6, 7, 0, 9, 1, 5, 4, 5, 5, 9, 8, 7, 9, 0, 3, 3, 6, 1, 6, 8, 7, 7, 9, 2, 6, 5, 2, 7, 5, 7, 3, 8, 0, 8, 3, 0, 3, 2, 8, 7, 9, 2, 0, 2, 1, 0, 0, 7, 8, 7, 9, 4, 2, 2, 9, 8, 6, 6, 8, 4, 9, 7, 4, 3, 5, 2, 2, 8, 2, 

Compare this to the previous predictions

In [0]:
db["ensemble"] = output
db.corr()

,DAB Drop Aug (98.88%),DAB Drop Aug full (98.2%),DAB opt SGD 2,DAB opt ADAM 1,DAB opt ADAM 2,AlexNet 7 Adam,AlexNet 7 SGD,AlexNet 7 Adam full,AlexNet 7 SGD full,Super Awesome Dodgie thing no pad,Super Awesome Dodgie thing pad,AlexNet 6 wide,ensemble
DAB Drop Aug (98.88%),1.000000,0.988160,0.987419,0.986022,0.986022,0.987594,0.988542,0.985701,0.984890,0.987295,0.984803,0.984671,0.989995
DAB Drop Aug full (98.2%),0.988160,1.000000,0.985661,0.983934,0.983934,0.985670,0.986916,0.981631,0.984927,0.985840,0.982116,0.980758,0.988737
DAB opt SGD 2,0.987419,0.985661,1.000000,0.983237,0.983237,0.984192,0.985791,0.984815,0.982747,0.987268,0.981633,0.979733,0.989688
DAB opt ADAM 1,0.986022,0.983934,0.983237,1.000000,1.000000,0.984980,0.985189,0.982190,0.984366,0.984201,0.982942,0.980670,0.989897
DAB opt ADAM 2,0.986022,0.983934,0.983237,1.000000,1.000000,0.984980,0.985189,0.982190,0.984366,0.984201,0.982942,0.980670,0.989897
AlexNet 7 Adam,0.987594,0.985670,0.984192,0.984980,0.984980,1.000000,0.987954,0.986296,0.985068,0.985386,0.980343,0.983232,0.992785
AlexNet 7 SGD,0.988542,0.986916,0.985791,0.985189,0.985189,0.987954,1.000000,0.987855,0.987465,0.985860,0.981396,0.984165,0.992534
AlexNet 7 Adam full,0.985701,0.981631,0.984815,0.982190,0.982190,0.986296,0.987855,1.000000,0.983851,0.984448,0.978563,0.982342,0.987745
AlexNet 7 SGD full,0.984890,0.984927,0.982747,0.984366,0.984366,0.985068,0.987465,0.983851,1.000000,0.984754,0.979847,0.981868,0.988131
Super Awesome Dodgie thing no pad,0.987295,0.985840,0.987268,0.984201,0.984201,0.985386,0.985860,0.984448,0.984754,1.000000,0.984643,0.979731,0.989139


Save this model

In [0]:
save_csv(output, "ensemble", "/content/gdrive/My Drive/models/")

successfully saved in /content/gdrive/My Drive/models/ensemble.csv


## Ensemble AlexNet7 Models

### Load Data

In [0]:
train_feat = np.load("/content/data/kmnist-train-imgs.npy")
train_targ = np.load("/content/data/kmnist-train-labels.npy")
test_feat = np.load("/content/data/kmnist-test-imgs.npy")

Load the models

In [0]:
mat_dict1 = torch.load("/content/models/results_AlexNet7_DropAug_ADAM/AlexNet7_DropAug_ADAM_175.pth")
mat_dict2 = torch.load("/content/models/results_AlexNet7_DropAug_SGD/AlexNet7_DropAug_200.pth")

make wrappers for each model

In [0]:
wrapper_1 = train_wrapper(mat_dict1["model"], None, None, None, device=device)
wrapper_2 = train_wrapper(mat_dict2["model"], None, None, None, device=device)
wrapper_1.transform = mat_dict1["transform"]
wrapper_2.transform = mat_dict2["transform"]

No transform found, test data must be normalised manually
No transform found, test data must be normalised manually


Get PDFs from these models and check they each sum to 1

In [0]:
pred1 = wrapper_1.evaluate(test_feat, True)
pred2 = wrapper_2.evaluate(test_feat, True)
pred1.shape, pred2.shaped

((10000, 10), (10000, 10))

In [0]:
log_pdfs = np.stack([pred1, pred2])
log_pdfs.shape

(2, 10000, 10)

In [0]:
pdfs = np.exp(log_pdfs)
pdfs.sum(axis=2)

array([[1.0000001, 1.       , 1.       , ..., 1.       , 1.       ,
        1.0000004],
       [1.0000006, 1.0000001, 1.       , ..., 1.0000002, 1.       ,
        0.9999992]], dtype=float32)

Combine these PDFs to an average and make a prediction by taking the max

In [0]:
avg_pdf = pdfs.mean(axis=0)
prdict_combo = avg_pdf.argmax(axis=1)
prdict_combo

array([2, 9, 3, ..., 9, 4, 2])

save the prediction

In [0]:
save_csv(prdict_combo, "AlexNeyt7_combo", path="/content/gdrive/My Drive/models/")

successfully saved in /content/gdrive/My Drive/models/AlexNeyt7_combo.csv
